In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import numpy as np
import wasp
import json
from itertools import groupby

In [3]:
creds = wasp.gdrive.get_credential()
anno_files = wasp.gdrive.list_drive_folder("1yw7DiEXa8oj3ArCxvJpuUkzfe50Eg4U0", creds)

In [4]:
anno_files

[('12U7BdGTXKsmHf90D2Ug_2rWv909XV2AZejiyl10U0SY', 'select_wsd_Jessy'),
 ('13TBiYh_cItqM9rjPXv2O_qOfwQV_xPf4YPlq4G0RvWM', 'select_wsd_Sabrina'),
 ('14nT8E_evfy3tMOj2ZmkN7ck1sFyr256fIRTtrHTYEWE', 'new_task_joy'),
 ('1kGvkzh7BMP31SGm_vfpgpoR6J5X8J8ubEmnNZYMJb3E', 'select_wsd_Taco'),
 ('1EoeyvBD8p4ksVFm4GPsof7scb9IFWK8kU-1DLSVCEd4', 'new_task_yongfu'),
 ('1uDig4ogH1gh3wxAIVj6BzO42yL8aQtixFzMRT59gdLM', 'select_wsd_Bobby'),
 ('1oMT1QxWfxiotlqrpSt8Kq2Adn0-ijHRJggjVCUKQoeM', 'model_test'),
 ('1jn4lb5D7HDm0Ni8-JYNxwKBf9doMzfr__WFX-edXGio', 'select_wsd_Nick')]

In [7]:
wasp.gdrive.get_sheet_name(anno_files[0][0], creds)

['字詞表', '題目']

In [10]:
item_pool = []
for anno_id, anno_name in anno_files:
    if anno_name == "model_test": continue
    print(anno_name)    
    sheet_names = wasp.gdrive.get_sheet_name(anno_id, creds)
    if '題目' in sheet_names:
        sheet_name = "題目"
    else:
        sheet_name = ""
    data = wasp.gdrive.retrieve_sheet(anno_id, creds, sheet_name)
    data = [x for x in data[1:] if len(x) == len(data[0])]
    grp_data = groupby(data, lambda x: x[1])
    option_data = {grp_idx: list(grp_opts) for grp_idx, grp_opts in grp_data}    
    for grp_id, grp_items in option_data.items():        
        questions = grp_items[0][4]
        qid = grp_items[0][1]
        opt_items = [x[5] for x in grp_items]        
        answer = [idx for idx, opt_x in enumerate(grp_items) if opt_x[6] == '1']
        if not answer:
            print("no answer found in " + grp_id)
            break
        item_pool.append({"questions": questions, "options": opt_items, 'ans':answer[0], "qid": qid, 'src': anno_name})            

select_wsd_Jessy
select_wsd_Sabrina
new_task_joy
select_wsd_Taco
new_task_yongfu
no answer found in 2
select_wsd_Bobby
model_test
select_wsd_Nick


In [16]:
wasp.install_cache_dir("wasp_items")
wasp_items_path = wasp.get_cache_path("wasp_items", "items.json")
with wasp_items_path.open("w", encoding="UTF-8") as fout:
    json.dump(item_pool, fout, indent=2, ensure_ascii=False)

In [85]:
option_data = {grp_idx: list(grp_opts) for grp_idx, grp_opts in grp_data}

In [91]:
item_pool = []
for grp_id, grp_items in option_data.items():        
    questions = grp_items[0][4]
    opt_items = [x[5] for x in grp_items]
    print(grp_items)
    answer = [idx for idx, opt_x in enumerate(grp_items) if opt_x[6] == '1']
    if not answer:
        print("no answer found in " + grp_id)
        break
    item_pool.append({"questions": questions, "options": opt_items, 'ans':answer[0]})
    break
        

[['1', '1', '看', '用眼睛察覺。', '甲：他們公司假日也都要上班，<看>到他那樣子我都怕了。\n乙：？', '小心你的肝', '1'], ['2', '1', '看', '用眼睛察覺。', '甲：他們公司假日也都要上班，<看>到他那樣子我都怕了。\n乙：？', '裡面什麼都沒有提', '0'], ['3', '1', '看', '用眼睛察覺。', '甲：他們公司假日也都要上班，<看>到他那樣子我都怕了。\n乙：？', '要走一段路才會到', '0'], ['4', '1', '看', '用眼睛察覺。', '甲：他們公司假日也都要上班，<看>到他那樣子我都怕了。\n乙：？', '要不要預約一下', '0']]


In [92]:
item_pool

[{'questions': '甲：他們公司假日也都要上班，<看>到他那樣子我都怕了。\n乙：？',
  'options': ['小心你的肝', '裡面什麼都沒有提', '要走一段路才會到', '要不要預約一下'],
  'ans': 0}]

In [3]:
question = "物理學和化學是自然科學"
options = [
    "這個問題沒有答案", 
    "今天天氣真好",
    "速度等於距離除以時間",
    "光的速度是個常數"
]

qs, opts = wasp.segment_words(question, options)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\seantyh\AppData\Local\Temp\jieba.cache
Loading model cost 0.676 seconds.
Prefix dict has been built succesfully.


In [4]:
qmat, opt_list = wasp.to_vectors(qs, opts)

In [5]:
qs

['物理學', '和', '化學', '是', '自然科學']

In [6]:
wasp.answer(question, options)

QaAnswer(avg_ans=3, avg_vec=array([0.96438912, 0.95225327, 0.95532407, 0.97664661]), max_ans=3, max_vec=[0.9439243035979984, 0.9592233666708333, 0.9309127015914954, 0.9925515085569134], segments={'questions': ['物理學', '和', '化學', '是', '自然科學'], 'options': [['這個', '問題', '沒有', '答案'], ['今天', '天氣', '真好'], ['速度', '等於', '距離', '除以', '時間'], ['光', '的', '速度', '是個', '常數']]})

# Main